In [1]:
import os
import pandas as pd
import requests
import json
import re

# set working dir
os.chdir('/home/reihsuanc/generic-real-estate-consulting-project-group-20/data/raw')

# read in data
houses_df = pd.read_csv("All_Houses_Scraped.csv")

# drop redundant cols
houses_df = houses_df.drop(['Unnamed: 0', 'desc'], axis = 1)

# 14804 entries total
len(houses_df)

# look at resulting dataset
houses_df.head()

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.11) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


,coordinates,cost_text,name,rooms
0,"[-37.9032708, 145.0770553]",$680,15 Fintonia Street Hughesdale VIC 3166,"['3 Beds', '2 Baths', '2 Parking']"
1,"[-37.9077745, 145.092738]",$650.00,41 Bishop Street Oakleigh VIC 3166,"['3 Beds', '2 Baths', '2 Parking']"
2,"[-37.8922004, 145.0899378]",$500,109/6 Dalgety Street Oakleigh VIC 3166,"['1 Bed', '1 Bath', '1 Parking']"
3,"[-37.8972564, 145.0871089]",$570,16/30 Swindon Road Oakleigh VIC 3166,"['3 Beds', '2 Baths', '2 Parking']"
4,"[-37.894307, 145.082706]",$475,1/14 Bletchley Road Hughesdale VIC 3166,"['2 Beds', '1 Bath', '1 Parking']"


In [2]:
# find number of night cases
def night_cases():
    n = 0
    for cost in houses_df['cost_text']: 
        if re.search(r'\d+[\s+\w+\s+]*night', cost):
            n += 1
    return n

# 23 night cases
night_cases()

23

In [3]:
# note theres a single fortnightly case at 7774th entry
i = 0
for cost in houses_df['cost_text']:
    i += 1
    if re.search(r'\d+[\s+]*Fortnightly', cost):
        print(i)

7774


In [4]:
# function to convert nightly prices to weekly

def night_to_week(df):
    # iterate thru entire cost_text column
    for cost in df['cost_text']:
        
        # to prevent calling regex on empty lists, cuz findall() returns empty list if no match is found
        if re.findall(r'\d+[\s+\w+\s+]*night', cost):

            # find nightly cases
            cur_str = re.findall(r'\d+[\s+\w+\s+]*night', cost)

            # extract cost amount
            cur_str_2 = re.findall(r'\d+', cur_str[0])

            # replace the value with weekly price (multiply by 7)
            df['cost_text'] = df['cost_text'].replace([cost], str(int(cur_str_2[0]) * 7))

    return df

In [5]:
# apply function
cleaned_df = night_to_week(houses_df)
cleaned_df.head()

,coordinates,cost_text,name,rooms
0,"[-37.9032708, 145.0770553]",$680,15 Fintonia Street Hughesdale VIC 3166,"['3 Beds', '2 Baths', '2 Parking']"
1,"[-37.9077745, 145.092738]",$650.00,41 Bishop Street Oakleigh VIC 3166,"['3 Beds', '2 Baths', '2 Parking']"
2,"[-37.8922004, 145.0899378]",$500,109/6 Dalgety Street Oakleigh VIC 3166,"['1 Bed', '1 Bath', '1 Parking']"
3,"[-37.8972564, 145.0871089]",$570,16/30 Swindon Road Oakleigh VIC 3166,"['3 Beds', '2 Baths', '2 Parking']"
4,"[-37.894307, 145.082706]",$475,1/14 Bletchley Road Hughesdale VIC 3166,"['2 Beds', '1 Bath', '1 Parking']"


In [6]:
# double check no more night exists in df
for cost in cleaned_df['cost_text']:
    if re.findall(r'\d+[\s+\w+\s+]*night', cost):
        print('night found')

In [7]:
# check some old cells to see if cost conversion is correct
# kinda stupid but just found random nightly case on excel and its index
# was 260 per night so 260 * 7 = 1820 so correct
cleaned_df.iloc[622,1]

'1820'